# Create ISA-API Investigation from Datascriptor Study Design configuration
# Observational study on Human students

In this notebook I will show you how you can use a study design configuration is JSON format as produce by datascriptor (https://gitlab.com/datascriptor/datascriptor) to generate a single-study ISA investigation and how you can then serialise it in JSON and tabular (i.e. CSV) format.

Or study design configuration consists of:
- a 4-arm study design. Each arm has 10 subjects
- subjects are human, male and female, of three age groups in years (6-10, 10-15, 15-18) 
- two phases: the observational phase and follow-up
- one sample type colllected: blood
- two assay types: 


## 1. Setup

Let's import all the required libraries

In [1]:
from time import time
import os
import json

In [2]:
## ISA-API related imports
from isatools.model import Investigation, Study

In [3]:
## ISA-API create mode related imports
from isatools.create.model import StudyDesign
from isatools.create.connectors import generate_study_design

# serializer from ISA Investigation to JSON
from isatools.isajson import ISAJSONEncoder

# ISA-Tab serialisation
from isatools import isatab

In [4]:
## ISA-API create mode related imports
from isatools.create import model
from isatools import isajson

## 2. Load the Study Design JSON configuration

First of all we load the study design configurator with all the specs defined above

In [5]:
with open(os.path.abspath(os.path.join(
    "config", "observational-study-students.json"
)), "r") as config_file:
    study_design_config = json.load(config_file)
study_design_config

{'_id': '606454b9d071027feb44ad9b',
 'name': "Student's study",
 'description': 'In an observational study, the investigator simply records observations and analyzes data, without administration of an intervention or alteration of the care people receive. These studies may focus on risk factors, natural history, variations in disease progression or disease treatment without delivering an intervention. They often assess specific health characteristics of the enrolled human subjects by collecting medical/dental history, exposure, or clinical data; obtaining biospecimens (e.g., for biomarker or genomic analyses); or obtaining photographic, radiographic or other images from research subjects.',
 'design': {'isPending': False,
  'subjectType': 'Homo sapiens',
  'subjectSize': 100,
  'designType': {'term': 'observational design',
   'id': 'NCIT:C147138',
   'iri': 'http://purl.obolibrary.org/obo/NCIT_C147138',
   'label': 'Study subjects do not receive treatments',
   'value': 'observational

## 3. Generate the ISA Study Design from the JSON configuration
To perform the conversion we just need to use the function `generate_isa_study_design()` (name possibly subject to change, should we drop the "isa" and "datascriptor" qualifiers?)

In [6]:
study_design = generate_study_design(study_design_config)
assert isinstance(study_design, StudyDesign)

## 4. Generate the ISA Study from the StudyDesign and embed it into an ISA Investigation

The `StudyDesign.generate_isa_study()` method returns the complete ISA-API `Study` object.

In [7]:
start = time()
study = study_design.generate_isa_study()
end = time()
print('The generation of the study from the design took {:.2f} s.'.format(end - start))
assert isinstance(study, Study)
investigation = Investigation(identifier='inv01', studies=[study])

The generation of the study from the design took 4.07 s.


## 5. Serialize and save the JSON representation of the generated ISA Investigation

In [8]:
start = time()
inv_json = json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': '))
end = time()
print('The JSON serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The JSON serialisation of the ISA investigation took 2.20 s.


In [9]:
directory = os.path.abspath(os.path.join('output', 'observational-study-students'))
os.makedirs(directory, exist_ok=True)
with open(os.path.abspath(os.path.join(directory, 'isa-investigation-observational-design-students.json')), 'w') as out_fp:
    json.dump(json.loads(inv_json), out_fp)

## 6. Dump the ISA Investigation to ISA-Tab

In [ ]:
start = time()
isatab.dump(investigation, directory)
end = time()
print('The Tab serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

2021-07-13 09:19:00,523 [INFO]: isatab.py(_all_end_to_end_paths:1131) >> [0, 1, 2, 3, 4, 5]
2021-07-13 09:19:03,980 [WARNING]: isatab.py(write_study_table_files:1194) >> [7, 6, 0, 9, 8, 11, 10, 13, 12, 15, 14, 17, 16, 19, 18, 21, 20, 23, 22, 25, 24, 27, 26, 29, 28, 31, 30, 33, 32, 35, 34, 37, 36, 39, 38, 41, 40, 43, 42, 45, 44, 47, 46, 49, 48, 51, 50, 53, 52, 55, 54, 57, 56, 59, 58, 61, 60, 63, 62, 65, 64, 67, 66, 69, 68, 71, 70, 73, 72, 75, 74, 77, 76, 79, 78, 81, 80, 83, 82, 85, 84, 87, 86, 89, 88, 91, 90, 93, 92, 95, 94, 97, 96, 99, 98, 101, 100, 103, 102, 105, 104, 107, 106, 109, 108, 111, 110, 113, 112, 115, 114, 117, 116, 119, 118, 121, 120, 123, 122, 125, 124, 127, 126, 129, 128, 131, 130, 133, 132, 135, 134, 137, 136, 139, 138, 141, 140, 143, 142, 145, 144, 147, 146, 149, 148, 151, 150, 153, 152, 155, 154, 157, 156, 159, 158, 161, 160, 163, 162, 165, 164, 167, 166, 169, 168, 171, 170, 173, 172, 175, 174, 177, 176, 179, 178, 181, 180, 183, 182, 185, 184, 187, 186, 189, 188, 191,

2021-07-13 09:19:03,998 [INFO]: isatab.py(_longest_path_and_attrs:1091) >> [[0, 7, 6], [0, 9, 8], [0, 11, 10], [0, 13, 12], [0, 15, 14], [0, 17, 16], [0, 19, 18], [0, 21, 20], [0, 23, 22], [0, 25, 24], [0, 27, 26], [0, 29, 28], [0, 31, 30], [0, 33, 32], [0, 35, 34], [0, 37, 36], [0, 39, 38], [0, 41, 40], [0, 43, 42], [0, 45, 44], [0, 47, 46], [0, 49, 48], [0, 51, 50], [0, 53, 52], [0, 55, 54], [0, 57, 56], [0, 59, 58], [0, 61, 60], [0, 63, 62], [0, 65, 64], [0, 67, 66], [0, 69, 68], [0, 71, 70], [0, 73, 72], [0, 75, 74], [0, 77, 76], [0, 79, 78], [0, 81, 80], [0, 83, 82], [0, 85, 84], [0, 87, 86], [0, 89, 88], [0, 91, 90], [0, 93, 92], [0, 95, 94], [0, 97, 96], [0, 99, 98], [0, 101, 100], [0, 103, 102], [0, 105, 104], [0, 107, 106], [0, 109, 108], [0, 111, 110], [0, 113, 112], [0, 115, 114], [0, 117, 116], [0, 119, 118], [0, 121, 120], [0, 123, 122], [0, 125, 124], [0, 127, 126], [0, 129, 128], [0, 131, 130], [0, 133, 132], [0, 135, 134], [0, 137, 136], [0, 139, 138], [0, 141, 140], [0

To use them on the notebook we can also dump the tables to pandas DataFrames, using the `dump_tables_to_dataframes` function rather than dump

In [ ]:
dataframes = isatab.dump_tables_to_dataframes(investigation)

In [ ]:
len(dataframes)

## 7. Check the correctness of the ISA-Tab DataFrames 

We have 1 study file and 2 assay files (one for MS and one for NMR). Let's check the names:

In [ ]:
for key in dataframes.keys():
    display(key)

### 7.1 Count of subjects and samples

We have 10 subjects in the each of the six arms for a total of 60 subjects. 5 blood samples per subject are collected (1 in treatment 1 phase, 1 in treatment, and 3 in the follow-up phase) for a total of 300 blood samples. These will undergo the NMR assay. We have 4 saliva samples per subject (1 during screen and 3 during follow-up) for a total of 240 saliva samples. These will undergo the "mass spcetrometry" assay.

In [ ]:
study_frame = dataframes['s_study_01.txt']
count_arm0_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP0' in el)])
count_arm1_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP1' in el)])
count_arm2_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP2' in el)])
count_arm3_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP3' in el)])
count_arm4_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP4' in el)])
count_arm5_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP5' in el)])
print("There are {} samples in the GRP0 arm (i.e. group)".format(count_arm0_samples))
print("There are {} samples in the GRP1 arm (i.e. group)".format(count_arm1_samples))
print("There are {} samples in the GRP2 arm (i.e. group)".format(count_arm2_samples))
print("There are {} samples in the GRP3 arm (i.e. group)".format(count_arm3_samples))
print("There are {} samples in the GRP4 arm (i.e. group)".format(count_arm4_samples))
print("There are {} samples in the GRP5 arm (i.e. group)".format(count_arm5_samples))

In [ ]:
study_frame

In [ ]:
dataframes['a_AT11_clinical-chemistry_marker-panel.txt']

In [ ]:
dataframes['a_AT8_genome-sequencing_nucleic-acid-sequencing.txt']

### Overview of the Clinical Chemistry Marker Panel

For the mass. spec. assay table, we have 160 (derma) samples, 160 extracts (1 per  sample, "polar" fraction), 320 labeled extracts (2 per extract, as "#replicates" is  2) and 640 mass spec protocols + 640 output files (2 per labeled extract as we do 1 technical replicate with 2 protocol parameter combinations `["Agilent QTQF 6510", "FIA", "positive mode"]` and `["Agilent QTQF 6510", "LC", "positive mode"]`).

In [ ]:
dataframes['a_AT11_clinical-chemistry_marker-panel.txt'].nunique(axis=0, dropna=True)

### Overview of the Genome Sequencing assay table

For the NMR assay table, we have 360 (blood) samples, 720 extracts (2 per  sample, a single replicate of the "supernatant" and "pellet" fractions) and 2880 NMR protocols + 4800 output files (4 per extract as we do 2 technical replicates with 2 protocol parameter combinations `["Bruker Avance II 1 GHz", "1D 1H NMR", "CPGM"]` and `["Bruker Avance II 1 GHz", "1D 1H NMR", "TOCSY"]`).

In [ ]:
dataframes['a_AT8_genome-sequencing_nucleic-acid-sequencing.txt'].nunique(axis=0, dropna=True)